In [1]:
import pickle 
import os 
import pandas as pd 
import mlflow
import uuid 
from sklearn.feature_extraction import DictVectorizer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import make_pipeline

In [51]:
year = 2021
month = 1
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

# RUN_ID = '33d91be4d8184963b8648d4419ef6507' 
RUN_ID = os.getenv('RUN_ID')

In [53]:
# Create artificial ride_ids
def generate_uuids(n):
    "Write a function to generate artificial ride_ids"
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

# Read dataframe 
def read_dataframe(filename: str):
    "Write a function to read and processing dataframe"
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df['ride_id'] = generate_uuids(len(df))
    return df


def prepare_dictionaries(df: pd.DataFrame):
    "Write a function to combined features and prepared data"
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [55]:
def load_model(run_id):
    "Write a function to load model from artifact"
    logged_model = f'mlartifacts/1/{RUN_ID}/artifacts/model'
    #logged_model = f'runs:/{RUN_ID}/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, run_id, output_file):
    "Write a function to run the model"
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)
    return df_result

In [57]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,e4732905-7bb7-40f4-981b-2bfb5f7d29b4,2021-01-01 00:15:56,43,151,3.933333,6.862711,-2.929378,33d91be4d8184963b8648d4419ef6507
1,15ee51d8-b8b6-44a3-a96c-ee1de14103c6,2021-01-01 00:25:59,166,239,8.750000,13.368721,-4.618721,33d91be4d8184963b8648d4419ef6507
2,cc0b34ce-9527-4ee1-9d57-c417736b73b5,2021-01-01 00:45:57,41,42,5.966667,6.360871,-0.394204,33d91be4d8184963b8648d4419ef6507
3,d0bc2b2a-acb6-4996-9359-577d2eca7e7a,2020-12-31 23:57:51,168,75,7.083333,11.824423,-4.741089,33d91be4d8184963b8648d4419ef6507
7,53dfc172-79b6-4da9-b617-b16b6b993a2a,2021-01-01 00:26:31,75,75,2.316667,3.389290,-1.072623,33d91be4d8184963b8648d4419ef6507
...,...,...,...,...,...,...,...,...
76513,72295a1d-f424-46c6-bdd5-c2d7d5869090,2021-01-31 21:38:00,81,90,38.000000,41.526829,-3.526829,33d91be4d8184963b8648d4419ef6507
76514,38a25ae2-66f5-4184-bc4a-d78d19739f0d,2021-01-31 22:43:00,35,213,38.000000,43.858974,-5.858974,33d91be4d8184963b8648d4419ef6507
76515,aafa5e55-e06c-4027-8a62-8cc0cfb0d5b1,2021-01-31 22:16:00,74,69,11.000000,14.436509,-3.436509,33d91be4d8184963b8648d4419ef6507
76516,642947af-173f-4886-a808-9e9ec785d39f,2021-01-31 23:10:00,168,215,27.000000,37.092622,-10.092622,33d91be4d8184963b8648d4419ef6507


In [61]:
!ls output/green 

2021-01.parquet 2021-02.parquet
